In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

df = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df['DEATH_EVENT'].unique()

In [ ]:
age_yes = df[df['DEATH_EVENT'] == 1].age
age = df.age

plt.figure(figsize=(8,6))
plt.xlabel('Age')
plt.ylabel('Death Event')
plt.hist([age_yes, age], label=['Death Event', 'Total per age'])
plt.legend()

In [ ]:
yes = df[df['DEATH_EVENT'] == 1]['DEATH_EVENT']
no = df[df['DEATH_EVENT'] == 0]['DEATH_EVENT']

print(len(no))
print(len(yes))

In [ ]:
plat = df[df['DEATH_EVENT'] == 1].platelets

plt.figure(figsize=(8,6))
plt.xlabel('Platelets')
plt.ylabel('Death Event')
plt.hist(plat, label=['Death Event'])
plt.legend()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df.corr())
plt.legend()

In [ ]:
c = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[c] = scaler.fit_transform(df[c]) 

In [ ]:
df.head()

In [ ]:
X = df.drop('DEATH_EVENT', axis='columns')
y = df['DEATH_EVENT']

from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X,y)

from sklearn.model_selection import train_test_split
X_tain, X_test, y_train, y_test = train_test_split(X_sm,y_sm)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_tain, y_train)

In [ ]:
lr_pred = lr.predict(X_test)
lr_pred

In [ ]:
lr.score(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report as cr
print(cr(y_test, lr_pred))

In [ ]:
import tensorflow as tf
from tensorflow import keras

ann = keras.Sequential([keras.layers.Dense(12, input_shape=(12,), activation='relu'),
                        keras.layers.Dense(12, activation='relu'),
                        keras.layers.Dense(1, activation='sigmoid')])
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_tain, y_train, epochs=100)

In [ ]:
ann.evaluate(X_test, y_test)

In [ ]:
pred = ann.predict(X_test)

ann_pred = []
for i in pred:
  if i > 0.5:
    ann_pred.append(1)
  else:
    ann_pred.append(0)

In [ ]:
print(cr(y_test, ann_pred))